# NetworkSet


## Introduction



The [NetworkSet](../api/networkSet.rst) object represents an unordered  set of networks. It 
provides  methods iterating and slicing the set, sorting by datetime, calculating statistical quantities, and displaying uncertainty bounds on plots. 

## Creating a [NetworkSet](../api/networkSet.rst)

Lets take a look in the `data/` folder, there are some redundant measurements of a network called `ro`,  which is a *radiating open* waveguide. 

```
ls data/ro*

-a----       14/02/2021     12:35           8031 ro,1.s1p
-a----       14/02/2021     12:35           8030 ro,2.s1p
-a----       14/02/2021     12:35           8031 ro,3.s1p
-a----       14/02/2021     12:35          46592 ro_spreadsheet.xls
```

The files `ro,1.s1p` , `ro,2.s1p`, ...  are redundant measurements on 
which we would like to calculate statistics using the [NetworkSet](../api/networkSet.rst)
class.

A [NetworkSet](../api/networkSet.rst) is created from a list or dict of 
[Network](../api/network.rst)'s. So first we need to load all of the 
touchstone files into `Networks`. This can be done quickly with 
`rf.read_all`,  The argument `contains` is used to load only files 
which match a given substring. 

In [ ]:
import skrf as rf

rf.read_all(rf.data.pwd, contains='ro')

This can be passed directly to the [NetworkSet](../api/networkSet.rst) constructor, 

In [ ]:
from skrf import NetworkSet 

ro_dict = rf.read_all(rf.data.pwd, contains='ro')
ro_ns = NetworkSet(ro_dict, name='ro set') 
ro_ns

A NetworkSet can also be constructed directly from:
 - a directory containing Touchstone files: `NetworkSet.from_dir()`,
 - a zipfile of touchstones files: `NetworkSet.from_zip()`, 
 - a dictionnary of s-parameters: `NetworkSet.from_s_dict()`,
 - a (G)MDIF (.mdf) file: `NetworkSet.from_mdif()`,
 - a CITI (.cti) file: `NetworkSet.from_citi()`. 

## Accessing Network Methods 
The [Network](../api/network.rst) elements in a [NetworkSet](../api/networkSet.rst) can be accessed like the elements of list, 

In [ ]:
ro_ns[0]

Most [Network](../api/network.rst) methods are also methods of 
[NetworkSet](../api/networkSet.rst). These methods are called on each 
[Network](../api/network.rst) element individually. For example to 
plot the log-magnitude of the s-parameters of each Network.

In [ ]:
%matplotlib inline
from pylab import *
import skrf as rf
rf.stylely()

ro_ns.plot_s_db()

## Statistical Properties


Statistical quantities can be calculated by accessing 
properties of the NetworkSet. To calculate the complex 
average of the set, access the `mean_s` property

In [ ]:
ro_ns.mean_s

    
The naming convention of the statistical operator properties are `NetworkSet.{function}_{parameter}`, where `function` is the name of the 
statistical function, and `parameter` is the Network parameter to operate 
on. These methods return a [Network](../api/network.rst) object, so they can be 
saved or plotted in the same way as you would with a Network.
To plot the log-magnitude of the complex mean response 

In [ ]:
ro_ns.mean_s.plot_s_db(label='ro')

Or to plot the standard deviation of the complex s-parameters,

In [ ]:
ro_ns.std_s.plot_s_re(y_label='Standard Deviations')

Using these properties it is possible to calculate statistical quantities on the scalar 
components of the complex network parameters. To calculate the 
mean of the phase component,

In [ ]:
ro_ns.mean_s_deg.plot_s_re()    

## Plotting Uncertainty Bounds


Uncertainty bounds can be plotted through the methods 

In [ ]:
ro_ns.plot_uncertainty_bounds_s_db()

In [ ]:
ro_ns.plot_uncertainty_bounds_s_deg()


## Reading and Writing

To write all [Network](../api/network.rst)s of a [NetworkSet](../api/networkSet.rst) out to individual touchstones,

In [ ]:
ro_ns.write_touchstone(dir='data/')

For temporary data storage, [NetworkSet](../api/networkSet.rst)s can be saved and read from disk 
using  the functions `rf.read` and `rf.write`

    

In [ ]:
rf.write('ro set.ns', ro_ns)

In [ ]:
ro_ns = rf.read('ro set.ns')
ro_ns

## Export to Excel, csv, or html

[NetworkSet](../api/networkSet.rst)s can also be exported to other filetypes. The format of the output; real/imag, mag/phase is adjustable, as is the output type; csv, excel, html. For example to export mag/phase for each network into an Excel spreadsheet for your boss[s]

In [ ]:
ro_ns.write_spreadsheet('data/ro_spreadsheet.xls', form='db')

More info on this can be found in the function, `skrf.io.general.network_2_spreadsheet`

## Named Parameters
If all the `Network` objects of a `NetworkSet` have a `params` property containing a dictionnary of the named parameters and values associated to each Network, it is possible to select the Networks corresponding to a subset of named parameters using the `.sel()` method. 

The following example illustrates this feature.

In [ ]:
# dummy named parameters and values 'a', 'X' and 'c' 
import numpy as np

params = [
        {'a':0, 'X':10, 'c':'A'},
        {'a':1, 'X':10, 'c':'A'},
        {'a':2, 'X':10, 'c':'A'},
        {'a':1, 'X':20, 'c':'A'},
        {'a':0, 'X':20, 'c':'A'},
        ]
# create a NetworkSet made of dummy Networks, each define for set of parameters 
freq1 = rf.Frequency(75, 110, 101, 'ghz')
ntwks_params = [rf.Network(frequency=freq1, s=np.random.rand(len(freq1),2,2), 
                               name=f'ntwk_{m}', comment=f'ntwk_{m}', params=params)
                            for (m, params) in enumerate(params) ]     
ns = rf.NetworkSet(ntwks_params)
print(ns)

Selecting the sub-NetworkSet matching scalar parameters can be made as:

In [ ]:
ns.sel({'a': 1})

In [ ]:
ns.sel({'a': 0, 'X': 10})

Selecting the sub-NetworkSet matching a range of parameters also works:

In [ ]:
ns.sel({'a': 0, 'X': [10,20]})

In [ ]:
ns.sel({'a': [0,1], 'X': [10,20]})

The various named parameter keys and values of the NetworkSet can be retrieved using the `dims` and `coords` properties: 

In [ ]:
ns.dims

In [ ]:
ns.coords

## Interpolating between the Networks of a NetworkSet
It is possible to create new Networks interpolated from the Networks contained in a `NetworkSet`. If no `params` properties have been defined for each Network of the NetworkSet, the `interpolate_from_network()` method can be used to specify a interpolating parameter. 

In [ ]:
param_x = [1, 2, 3]  # a parameter associated to each Network
x0 = 1.5  # parameter value to interpolate for
interp_ntwk = ro_ns.interpolate_from_network(param_x, x0)
print(interp_ntwk)

An illustrated example is given in the [Examples section](../examples/index.rst#networksets) of the documentation.

It is also possible to interpolate using a named parameter when they have been defined: 

In [ ]:
# Interpolated Network for a=1.2 within X=10 Networks:
ns.interpolate_from_params('a', 1.2, {'X': 10})